In [ ]:
from pca import PCA_Analysis
from dotenv import load_dotenv
import os
import pandas as pd
import glob
from tqdm import tqdm

load_dotenv()
directory = os.getenv("path")
pca = PCA_Analysis()

Helper functions

In [5]:
# selecting top attributes from computed pca
def select_top_attr(all_top_attr, attribute) -> list:
    selected_attr = []
    top_attr = all_top_attr[attribute]
    attrs_list = list(top_attr.values())

    for i in range(4):
        selected_attr.append(attrs_list[0][i])
    for i in range(2):
        selected_attr.append(attrs_list[1][i])
    return list(set(selected_attr))

structural attributes (Time-invariant)

In [ ]:
def structural_pca() -> dict:
    # struct_path = os.path.join(directory, "data","raw_datasets", "structural_attributes", "140_TI_variables")
    struct_path = os.path.join(directory, "computed_data", "TI_variables")

    stations_list = pd.read_csv(os.path.join(directory, "raw_data", "stations_list.csv")).stations.tolist()

    struct_files = os.listdir(struct_path)
    struct_files = [file for file in struct_files if file.endswith(".csv")]
    struct_files.remove('leb_surficial.csv')


    all_top_load_dfs, all_top_attr = [], {}

    for file in struct_files:
        attr_type = file.split("_")[1].split(".")[0]
        input_struct_data = pd.read_csv(os.path.join(struct_path, file))

        input_struct_data = input_struct_data[input_struct_data["station_id"].isin(stations_list)]


        struct_data = pd.concat([input_struct_data["station_id"], input_struct_data.iloc[:, 4:]], axis = 1)
        struct_data = struct_data.set_index("station_id")

        loadings = pca.loadings(struct_data)

        explained_var = pca.explained_variance(struct_data)
        pc1_val = round(explained_var["Explained Variance"].iloc[1],2)
        pc2_val = round(explained_var["Explained Variance"].iloc[2],2)

        print(f'explained variance for {attr_type} PC1: {pc1_val} and PC2: {pc2_val} summed up to {pc1_val + pc2_val}')

        top_attr = pca.top_attributes(loadings, 5)
        all_top_attr[attr_type] = top_attr
        new_keys = {'PC1': f'{attr_type}_PC1_{pc1_val}', 'PC2': f'{attr_type}_PC2_{pc2_val}'}
        renamed_top_attr = {new_keys[key]: value for key, value in top_attr.items()}
        top_load_df = pd.DataFrame(renamed_top_attr)
        all_top_load_dfs.append(top_load_df)
    # return  pd.concat(all_top_load_dfs, axis = 1)
    # out_put_df.to_csv(os.path.join(directory, "pca_results", "TI_top_attributes.csv"), index=False)
    return all_top_attr

structural_pca()

In [ ]:
# Generate new datasets with selected attributes
def generate_struct_df() -> pd.DataFrame:
    attr_collection = []
    struct_pca = structural_pca()
    for key in struct_pca.keys():
        attributes = select_top_attr(struct_pca, key)
        path = os.path.join(
            directory,
            "computed_data",
            "TI_variables",
            f"leb_{key}.csv",
        )
        stations_list = pd.read_csv(os.path.join(directory, "raw_data", "stations_list.csv")).stations.tolist()
        # input_struct_data = input_struct_data[input_struct_data["station_id"].isin(stations_list)]

        att_file = pd.read_csv(path)
        _133_stations_df = att_file[att_file["station_id"].isin(stations_list)]
        _133_stations_df = _133_stations_df[attributes]
        attr_collection.append(_133_stations_df)

    return pd.concat(attr_collection, axis=1).reset_index(drop=True)

generate_struct_df().columns

crop inventories (Time-variable)

In [ ]:
def crop_pca() -> dict:
    crops_path = os.path.join(directory, "computed_data", "crop_data")

    crop_inventories_files = glob.glob(f"{crops_path}/*.csv")

    all_crop_top_load_dfs, all_top_attr  = [], {}

    for path in crop_inventories_files:
        crop_yr_df = pd.read_csv(path)
        crop_yr_df = crop_yr_df.set_index(crop_yr_df.columns[0])

        # crop pca analysis
        # pca_df = pca.pca_analysis(crop_yr_df)
        loadings = pca.loadings(crop_yr_df)

        explained_var = pca.explained_variance(crop_yr_df)
        pc1_val = round(explained_var["Explained Variance"].iloc[1],2)
        pc2_val = round(explained_var["Explained Variance"].iloc[2],2)

        top_attr = pca.top_attributes(loadings, 5)
        all_top_attr[path.split("/")[-1].split(".")[0]] = top_attr
        new_keys = {'PC1': f'{path.split("/")[-1].split(".")[0]}_PC1_{pc1_val}', 'PC2': f'{path.split("/")[-1].split(".")[0]}_PC2_{pc2_val}'}
        renamed_top_attr = {new_keys[key]: value for key, value in top_attr.items()}
        top_load_df = pd.DataFrame(renamed_top_attr)
        all_crop_top_load_dfs.append(top_load_df)

    # crop_out_put_df = pd.concat(all_crop_top_load_dfs, axis = 1)
    # crop_out_put_df = crop_out_put_df.reindex(sorted(crop_out_put_df.columns), axis=1)
    # crop_out_put_df.to_csv(os.path.join(directory, "pca_results", "crop_top_attributes.csv"), index=False)

    return all_top_attr

print(crop_pca())

Combining TI (soil, lucl, terrain) and TV (crop inventory) datasets 

In [43]:
# helper function to select top attributes for year variant attributes.
def selectTopAtttributesForYear(datatype: dict) -> list:
    pca_selected_attr = []
    for _,val in datatype.items():
        yearTopAttributes, topAttributes = [], []
        for _,v in val.items():
            yearTopAttributes.append(v)

        for i in range(4):
            topAttributes.append(yearTopAttributes[0][i])
        for i in range(2):
            topAttributes.append(yearTopAttributes[1][i])

        pca_selected_attr.append(list(set(topAttributes)))

    # selected_attr = []
    counter = {}
    for arr_attrs in pca_selected_attr:
        for attr in arr_attrs:
            attr = attr.split("2")[0]
            if attr in counter:
                counter[attr] += 1
            else:
                counter[attr] = 1

    count, selected_attr = 0, set()
    count_values = sorted(list(counter.values()), reverse=True)
    # print("counter_values", count_values)
    lookup_counter = min(count_values[:5])

    for key, value in counter.items():
        if value >= lookup_counter:
            count += 1
            selected_attr.add(key)

    return list(selected_attr)

# testing:
# crops_data = crop_pca()

In [ ]:
# Generate crop + struct attributes combined dataset.
def generate_struct_crop_df(year:int) -> pd.DataFrame:
    crops_pca = crop_pca()
    selectedAttributesCrops = selectTopAtttributesForYear(crops_pca)
    selectedAttributesCropsWithYear = [0]*len(selectedAttributesCrops)

    for i in range(len(selectedAttributesCrops)):
        selectedAttributesCropsWithYear[i] = f"{selectedAttributesCrops[i]}{year}"

    for key in crops_pca.keys():
        if year == int(key.split("_")[-1]):
            # attributes = select_top_attr(crops_pca, key)
            path = os.path.join(directory, "computed_data","crop_data", f"{key}.csv")
            attr_file = pd.read_csv(path)
            attr_file = attr_file[selectedAttributesCropsWithYear]
            attr_file.columns = selectedAttributesCrops

            output_df = pd.concat([generate_struct_df(), attr_file], axis=1)
        else:
            pass

    return output_df

generate_struct_crop_df(2015).columns

feature engineer (functional attributes - riverflow metrics)

In [ ]:
# metrics to drop.
drop_metrics = [
    "Station Name",
    "Country",
    "Watershed-Area",
    "Latitude",
    "Longitude",
    "YR-MaxFlow",
    "GM-MaxFlow",
    "NGM-MaxFlow",
    "YR-MinFlow",
    "GM-MinFlow",
    "NGM-MinFlow",
    "YR-MedianFlow",
    "GM-MedianFlow",
    "NGM-MedianFlow",
    "YR-Q95Flow",
    "GM-Q95Flow",
    "NGM-Q95Flow",
    "YR-Q5Flow",
    "GM-Q5Flow",
    "NGM-Q5Flow",
]

def functional_pca(drop_metrics: list = drop_metrics):
    functional_path = os.path.join(directory, "computed_data", "flow_data")
    functional_files = glob.glob(f"{functional_path}/*.csv")

    all_top_load_dfs, all_top_metrics = [], {}

    for file in functional_files:
        df = pd.read_csv(file)
        df = df.set_index(df.columns[0])

        # drop metrics and checking for missing values.
        functional_df = df.loc[:, ~df.columns.str.startswith(tuple(drop_metrics))]
        functional_df = functional_df.fillna(functional_df.median())

        pca_loadings = pca.loadings(functional_df)

        explained_var = pca.explained_variance(functional_df)
        pc1_val = round(explained_var["Explained Variance"].iloc[1],2)
        pc2_val = round(explained_var["Explained Variance"].iloc[2],2)

        # print(f'variance for flow metrics {pc1_val + pc2_val}')

        top_metrics = pca.top_attributes(pca_loadings, 5)
        all_top_metrics[file.split("/")[-1].split(".")[0]] = top_metrics
        new_keys = {'PC1': f'{file.split("/")[-1].split(".")[0]}_PC1_{pc1_val}', 'PC2': f'{file.split("/")[-1].split(".")[0]}_PC2_{pc2_val}'}
        renamed_top_metrics = {new_keys[key]: value for key, value in top_metrics.items()}
        yr_top_load_df = pd.DataFrame(renamed_top_metrics)
        all_top_load_dfs.append(yr_top_load_df)

    # out_put_df = pd.concat(all_top_load_dfs, axis = 1)
    # out_put_df =out_put_df.reindex(sorted(out_put_df.columns), axis=1)

    return all_top_metrics

functionalAttributes = functional_pca()
# print(selectTopAtttributesForYear(functionalAttributes))
print(functionalAttributes)


In [ ]:
# pca for climate indices
def climate_pca():
    climate_path = os.path.join(directory, "computed_data", "climate_data")
    climate_files = glob.glob(f"{climate_path}/*.csv")

    all_top_load_dfs, all_top_metrics = [], {}

    for file in climate_files:
        df = pd.read_csv(file)

        stations_list = pd.read_csv(os.path.join(directory, "raw_data", "stations_list.csv")).stations.tolist()
        stations_list = sorted(stations_list)

        df.columns.values[0] = "station_id"
        df['station_id'] = df['station_id'].apply(lambda val: val.lstrip("0") if val.startswith("04") else val)

        climate_df = df[df["station_id"].isin(stations_list)]

        # sort the climate_df
        climate_df = climate_df.sort_values("station_id")
        climate_df.index = range(len(climate_df))

        climate_df = climate_df.drop(columns=["station_id"])

        # drop metrics and checking for missing values.
        climatedfNomissingValues = climate_df.fillna(climate_df.median())

        pca_loadings = pca.loadings(climatedfNomissingValues)

        explained_var = pca.explained_variance(climatedfNomissingValues)
        pc1_val = round(explained_var["Explained Variance"].iloc[1],2)
        pc2_val = round(explained_var["Explained Variance"].iloc[2],2)

        # print(f'variance for climate {pc1_val + pc2_val}')

        top_metrics = pca.top_attributes(pca_loadings, 5)
        all_top_metrics[file.split("/")[-1].split(".")[0]] = top_metrics
        new_keys = {'PC1': f'{file.split("/")[-1].split(".")[0]}_PC1_{pc1_val}', 'PC2': f'{file.split("/")[-1].split(".")[0]}_PC2_{pc2_val}'}
        renamed_top_metrics = {new_keys[key]: value for key, value in top_metrics.items()}
        yr_top_load_df = pd.DataFrame(renamed_top_metrics)
        all_top_load_dfs.append(yr_top_load_df)

        out_put_df = pd.concat(all_top_load_dfs, axis = 1)
        out_put_df =out_put_df.reindex(sorted(out_put_df.columns), axis=1)

    return all_top_metrics

climateAttributes = climate_pca()
print(selectTopAtttributesForYear(climateAttributes))

Generating Datasets

In [ ]:
# generate new dataset for AP classification (riverflows metrics - year and seasonal).
def selected_func_metrics() -> dict:

    stations_list = pd.read_csv(os.path.join(directory, "raw_data", "stations_list.csv")).stations.tolist()

    func_metrics = {}

    # riveflow metrics:
    functionalAttributes = selectTopAtttributesForYear(functional_pca())
    for year in range(2011, 2021):
        functionalAttributesWithYear = [functionalAttributes[i] + str(year) for i in range(len(functionalAttributes))]

        functional_path = os.path.join(directory, "computed_data", "flow_data", f'metrics{year}.csv')
        functional_df = pd.read_csv(functional_path)
        functional_df = functional_df[functionalAttributesWithYear]
        functional_df['station_id'] = stations_list
        functional_df = functional_df[["station_id"] + [col for col in functional_df.columns if col != "station_id"]]
        func_metrics[str(year)] = functional_df
        dir = os.path.join(directory, "computed_data", "pca_results","func")
        # functional_df.to_csv(dir +  f"/{year}_func_metrics.csv", index=False)

    return functional_df

selected_func_metrics()

In [ ]:
# generate new dataset for AP classification (riverflows metrics - climate indices).
def selected_func_climate_attrs() -> pd.DataFrame:
    climateAttributes = selectTopAtttributesForYear(climate_pca())

    for year in tqdm(range(2011, 2021)):
        df = pd.read_csv(
            os.path.join(
                directory,
                "computed_data",
                "climate_data",
                f"climate_indices_{str(year)}.csv",
            )
        )

        stations_list = pd.read_csv(os.path.join(directory, "raw_data", "stations_list.csv")).stations.tolist()

        stations_list = sorted(stations_list)

        df.columns.values[0] = "station_id"
        df['station_id'] = df['station_id'].apply(lambda val: val.lstrip("0") if val.startswith("04") else val)

        climate_df = df[df["station_id"].isin(stations_list)]

        # sort the climate_df
        climate_df = climate_df.sort_values("station_id")
        climate_df.index = range(len(climate_df))

        climateAttributesWithYear = [climateAttributes[i] + str(year) for i in range(len(climateAttributes))]
        climate_df = climate_df[climateAttributesWithYear]

        dir = os.path.join(directory, "computed_data", "pca_results", "func", f"{year}_func_metrics.csv")
        functional_df = pd.read_csv(dir)

        output_df = pd.concat([functional_df, climate_df], axis=1)

        output_df = output_df[["station_id"] + [col for col in output_df.columns if col != "station_id"]]

        # output_df.to_csv(os.path.join(output_dir, "pca_results", "func_climate", f"{year}_func_climate_attrs.csv"), index=False)

    return output_df

selected_func_climate_attrs()

In [ ]:
# generate new dataset for AP classification (riverflow metrics + structural attributes + climate indices)
stations_list = pd.read_csv(os.path.join(directory, "raw_data", "stations_list.csv")).stations.tolist()

stations_list = sorted(stations_list)


for year in tqdm(range(2011, 2021)):
    df = pd.read_csv(os.path.join(directory,"computed_data", "pca_results","func_climate", f"{year}_func_climate_attrs.csv"))
    output_df = pd.concat([generate_struct_crop_df(year), df], axis=1)
    output_df['station_id'] = stations_list
    output_df = output_df[["station_id"] + [col for col in output_df.columns if col != "station_id"]]
    # output_df.to_csv(os.path.join(directory,"computed_data", "pca_results","all_attributes", f"{year}_func_struct_climate_attrs.csv"), index=False)
output_df.columns

PCA TESTING and AP clustering for Flow metrics.

In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# metrics to drop.
drop_metrics = [
    "Station Name",
    "Country",
    "Watershed-Area",
    "Latitude",
    "Longitude",
    "YR-MaxFlow",
    "GM-MaxFlow",
    "NGM-MaxFlow",
    "YR-MinFlow",
    "GM-MinFlow",
    "NGM-MinFlow",
    "YR-MedianFlow",
    "GM-MedianFlow",
    "NGM-MedianFlow",
    "YR-Q95Flow",
    "GM-Q95Flow",
    "NGM-Q95Flow",
    "YR-Q5Flow",
    "GM-Q5Flow",
    "NGM-Q5Flow",
]


functional_path = os.path.join(directory, "computed_data", "flow_data", "metrics2011.csv")
functional_df = pd.read_csv(functional_path)
functional_df = functional_df.set_index(functional_df.columns[0])
functional_df = functional_df.loc[:, ~functional_df.columns.str.startswith(tuple(drop_metrics))]
functional_df = functional_df.fillna(functional_df.median())

scaler = StandardScaler()
scaled_df = scaler.fit_transform(functional_df)
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(scaled_df)

principalDf = pd.DataFrame(data = principalComponents, columns = ['PC1', 'PC2'])

# explained_variance = pca.explained_variance_ratio_
explained_variance_ratio = pca.explained_variance_ratio_
# print("Explained Variance Ratio:", explained_variance_ratio)


cumulative_variance_ratio = explained_variance_ratio.cumsum()
n_components = len(cumulative_variance_ratio[cumulative_variance_ratio <= 0.95])
# print("Number of Principal Components to retain 95% variance:", n_components)


# loadings
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

loadings_df = pd.DataFrame(loadings, columns = ['PC1', 'PC2'], index = functional_df.columns)

pc1 = loadings_df['PC1']


# use the value of pc1 to get the top 5 attributes
topLoadingAttributes_pc1 = pc1.abs().sort_values(ascending=False).head(5).index
topLoadingAttributes_pc2 = loadings_df['PC2'].abs().sort_values(ascending=False).head(2).index

# topLoadingAttributes = list(set(topLoadingAttributes_pc1).union(set(topLoadingAttributes_pc2)))
topLoadingAttributes = [
        "YR-RBI-2011",
        "NGM-RBI-2011",
        "Specific-GM-MedianFlow-2011",
        "Specific-NGM-MedianFlow-2011",
        "YR-CVQ-2011",
        "Specific-YR-MedianFlow-2011",
        "Specific-GM-Q95-2011",
        ]


resultedTopAttributes_df = functional_df[topLoadingAttributes]

# AP
from sklearn.cluster import AffinityPropagation
import warnings
warnings.filterwarnings("ignore")
model = AffinityPropagation(damping=0.9, verbose=2)

# fit the model
resultedTopAttributes_df = resultedTopAttributes_df.fillna(resultedTopAttributes_df.median())
model.fit(resultedTopAttributes_df)
labels = model.labels_

ap_res = {}
ap_res[functional_path.split("/")[-1].split(".")[0]] = list(labels)
result = pd.DataFrame(ap_res).set_index(resultedTopAttributes_df.index)

# min and max values of the labels
print("Min Label:", min(labels), "Max Label:", max(labels))

Converged after 52 iterations.
Min Label: 0 Max Label: 10
